# Design

This is an decoder model which will take in a roundness value and output a pseudoword that corresponds to the roundness value

In [1]:
from utils.pseudoword_generator import *
import pandas as pd
import torch

pd.set_option('display.max_columns', None)
device = "cuda" if torch.cuda.is_available() else "cpu"
state = 42

VERSION = 2

c:\Users\qkm20\anaconda3\envs\pseudoword\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
# Import dataset

data = pd.read_csv('datasets/japanese_pseudowords.csv')
data

,Pseudoword,Roundness
0,toyaun,0.025297
1,bineribuu,0.999838
2,zafugaho,0.984310
3,podo,0.191057
4,nteno,0.796121
...,...,...
9995,jitakupore,0.998109
9996,gokeza,0.137229
9997,mana,0.403694
9998,sabizamon,0.999407


In [3]:
data.describe()

,Roundness
count,10000.000000
mean,0.692622
std,0.317500
min,0.004033
25%,0.435884
50%,0.797849
75%,0.993706
max,1.000000


In [4]:
# Split data into train, val and test sets

trn = data.sample(frac=0.8, random_state=state)
val = data.drop(trn.index).sample(frac=0.5, random_state=state)
tst = data.drop(trn.index).drop(val.index)
trn.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)
tst.reset_index(inplace=True, drop=True)

In [5]:
print(f"Train set: {len(trn)} samples, Validation set: {len(val)} samples, Test set: {len(tst)} samples")

Train set: 8000 samples, Validation set: 1000 samples, Test set: 1000 samples


# Model

In [6]:
model = RoundnessToTextModel(
    t5_model_name="sonoisa/t5-base-japanese",
    freeze_t5=False,
    hidden_dim=256,
    output_dim=768,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

In [7]:
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.6)

# Training

In [8]:
train(
    model=model,
    optimizer=optimizer,
    trn_roundness=trn["Roundness"],
    val_roundness=val["Roundness"],
    tst_roundness=tst["Roundness"],
    trn_texts=trn["Pseudoword"],
    val_texts=val["Pseudoword"],
    tst_texts=tst["Pseudoword"],
    batch_size=min(len(val), 100),
    epochs=100,
    patience=10,
    scheduler=scheduler,
)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch   1/100, Train Loss: 8.7043, Validation Loss: 3.3928
Epoch   2/100, Train Loss: 3.6445, Validation Loss: 3.3851
Epoch   3/100, Train Loss: 3.5376, Validation Loss: 3.3085
Epoch   4/100, Train Loss: 3.4512, Validation Loss: 3.5099
Epoch   5/100, Train Loss: 3.4278, Validation Loss: 3.4331
Epoch   6/100, Train Loss: 3.4159, Validation Loss: 3.4842
Epoch   7/100, Train Loss: 3.4047, Validation Loss: 3.3863
Epoch   8/100, Train Loss: 3.3859, Validation Loss: 3.3795
Epoch   9/100, Train Loss: 3.3766, Validation Loss: 3.3940
Epoch  10/100, Train Loss: 3.3598, Validation Loss: 3.3772
Epoch  11/100, Train Loss: 3.3618, Validation Loss: 3.4221
Epoch  12/100, Train Loss: 3.3526, Validation Loss: 3.3451
Epoch  13/100, Train Loss: 3.3446, Validation Loss: 3.3775
Early stopping triggered after 13 epochs
Test Loss: 3.4649


# Testing

In [9]:
random_sample = tst.sample(n=10, random_state=42)
for _, row in random_sample.iterrows():
    print(f"Roundness Value : {row['Roundness']}")
    print(f"Original Word   : {row['Pseudoword']}")
    print(f"Predicted word  : {inference(model, row["Roundness"])}")
    print()

Roundness Value : 0.39937696
Original Word   : aya
Predicted word  : zeki

Roundness Value : 0.9291309
Original Word   : torechitsugo
Predicted word  : hireneo

Roundness Value : 0.9994641
Original Word   : sanumuzayo
Predicted word  : gutoi

Roundness Value : 0.9977919
Original Word   : hegaefumu
Predicted word  : zohoshioge

Roundness Value : 0.5365084
Original Word   : nopune
Predicted word  : shinabe

Roundness Value : 0.17295608
Original Word   : roji
Predicted word  : hakae

Roundness Value : 0.99980015
Original Word   : dabupiyome
Predicted word  : kepibugi

Roundness Value : 0.9962239
Original Word   : gobapawa
Predicted word  : pikuyue

Roundness Value : 0.58562106
Original Word   : yonyu
Predicted word  : mobewa

Roundness Value : 0.5903554
Original Word   : kechi
Predicted word  : sahe



In [10]:
roundness_list = []
for i in range(11):
    roundness_list.append(i/10)

for roundness in roundness_list:
    print(f"Roundness Value: {roundness}")
    print(f"Predicted word: {inference(model, roundness)}")
    print()

Roundness Value: 0.0
Predicted word: mohi

Roundness Value: 0.1
Predicted word: mone

Roundness Value: 0.2
Predicted word: sau

Roundness Value: 0.3
Predicted word: ugo

Roundness Value: 0.4
Predicted word: kikige

Roundness Value: 0.5
Predicted word: kote

Roundness Value: 0.6
Predicted word: ragu

Roundness Value: 0.7
Predicted word: pebi

Roundness Value: 0.8
Predicted word: yogae

Roundness Value: 0.9
Predicted word: aruso

Roundness Value: 1.0
Predicted word: fusozu



# Save and load model

In [11]:
save_model(model, filename=f"pseudoword_generator_v0{VERSION}.pth")

Model saved to outputs/pseudoword_generator_v02.pth


In [ ]:
model = load_model(filename=f"pseudoword_generator_v0{VERSION}.pth")